In [1]:
import os

In [2]:
!pwd

/Users/dhruvshah/Desktop/MLOps/ML-AWS/NLP_Project/experiments


In [3]:
os.chdir("../")
!pwd

/Users/dhruvshah/Desktop/MLOps/ML-AWS/NLP_Project


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    """
    Configuration for data ingestion.
    """
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directory


In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path: str = CONFIG_FILE_PATH,
                 params_file_path: str = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        lst = [self.config.artifacts_root]
        create_directory(lst)
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directory([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [7]:
import urllib.request as request
import zipfile
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size

In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info(f"Downloading data from {self.config.source_url}")
            filename, headers = request.urlretrieve(
                url = self.config.source_url, 
                filename = self.config.local_data_file)
            logger.info(f"{filename} downloaded with information: \n{headers}")
        else:
            logger.info(f"{self.config.local_data_file} already exists")

    def extract_data(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_obj:
            print(zip_obj)
            zip_obj.extractall(unzip_path)


In [18]:

config_manager = ConfigurationManager()
data_ingestion_config = config_manager.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)
data_ingestion.download_data()
print(data_ingestion_config.local_data_file)
data_ingestion.extract_data()

2024-04-04 00:00:57,575 - INFO - common - Yaml file config/config.yaml read successfully
2024-04-04 00:00:57,577 - INFO - common - Yaml file params.yaml read successfully
2024-04-04 00:00:57,578 - INFO - common - Directory artifacts created
2024-04-04 00:00:57,578 - INFO - common - Directory artifacts/data_ingestion created
2024-04-04 00:00:57,579 - INFO - 3596086911 - artifacts/data_ingestion/data.zip already exists
artifacts/data_ingestion/data.zip


BadZipFile: File is not a zip file